In [2]:
from map import EMmap, unify_dims
import os
from enum import Enum
from fitter import MapFitter
import torch

In [3]:
class Mode(Enum):
    V = "VecProduct"
    O = "Overlap"
    C = "CC"
    P = "PCC"
    L = "Laplacian"

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
device = torch.device("cuda:0")

In [5]:
ref_map = EMmap("/net/kihara-fast-scratch/wang3702/DiffModeler/Predict_Result/11237/structure_assembling/iterative_M/segment_map_small.mrc")
tgt_map = EMmap("/net/kihara-fast-scratch/wang3702/DiffModeler/Predict_Result/11237/structure_assembling/iterative_M/simumap_backbone_1A.mrc")

/home/kihara/zhu773/.virtualenvs/VESPER_CUDA/lib/python3.10/site-packages/mrcfile/mrcinterpreter.py:219: RuntimeWarning: Unrecognised machine stamp: 0x00 0x00 0x44 0x44
  warnings.warn(str(err), RuntimeWarning)


In [6]:
modeVal = Mode["C"].value

In [7]:
ref_map.set_vox_size(thr=0.1, voxel_size=2)
tgt_map.set_vox_size(thr=0.01, voxel_size=2)


#dmax=39.131189606246316
Nvox= 39, 39, 39
cent= 420.0, 461.5, 361.0
ori= 381.0, 422.5, 322.0

#dmax=36.96281915655244
Nvox= 36, 36, 36
cent= 422.5, 461.0, 360.5
ori= 386.5, 425.0, 324.5


In [8]:
unify_dims([ref_map, tgt_map], voxel_size=2)

In [9]:
# resample the maps using mean-shift with Gaussian kernel and calculate the vector representation
print("\n###Processing Reference Map Resampling###")
ref_map.resample_and_vec(dreso=2)
print("\n###Processing Target Map Resampling###")
tgt_map.resample_and_vec(dreso=2)
print()


###Processing Reference Map Resampling###
#maxd= 4.0
#fsiv= 1.0
#MAP SUM=5123.80419921875 COUNT=2220 AVE=2.3080198764801025 STD=112.5767822265625 STD_norm=29.115114212036133

###Processing Target Map Resampling###
#maxd= 4.0
#fsiv= 1.0
#MAP SUM=6956.66455078125 COUNT=989 AVE=7.034039497375488 STD=274.8923645019531 STD_norm=163.1942596435547



In [10]:
fitter = MapFitter(
            ref_map,
            tgt_map,
            10,
            modeVal,
            True,
            "/net/kihara-fast-scratch/wang3702/DiffModeler/Predict_Result/11237/structure_modeling/LDP_2/chain_LDP.pdb",
            "/net/kihara-fast-scratch/wang3702/DiffModeler/Predict_Result/11237/structure_assembling/iterative_M/backbone.pdb",
            "/net/kihara-fast-scratch/wang3702/DiffModeler/Predict_Result/11237/structure_modeling/M/fit_experiment_0/PDB/#0.pdb",
            2,
            True,
            device,
            10,
            None,
            None,
            None,
            None,
        )

###Initializing fitter###


In [ ]:
%load_ext line_profiler

In [ ]:
fitter._rot_and_search_fft((30,30,30), False)

In [ ]:
%%timeit
fitter._rot_and_search_fft((30,30,30), False)

In [ ]:
%lprun -f fitter._rot_and_search_fft fitter._rot_and_search_fft((30,30,30), False)

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
rot_mtx = R.from_euler("xyz", (30,30,30), degrees=True).as_matrix().astype(np.float32)

In [ ]:
rot_mtx = torch.from_numpy(rot_mtx).to(device)

In [ ]:
%%timeit
fitter._gpu_rot_map(fitter.tgt_map.new_data_gpu,fitter.tgt_map.vec_gpu,rot_mtx,fitter.tgt_map.search_pos_grid_gpu,fitter.device,rot_vec=(fitter.mode == "VecProduct"),)

In [ ]:
%lprun -f fitter._gpu_rot_map fitter._gpu_rot_map(fitter.tgt_map.new_data_gpu,fitter.tgt_map.vec_gpu,rot_mtx,fitter.tgt_map.search_pos_grid_gpu,fitter.device,rot_vec=(fitter.mode == "VecProduct"),)

In [ ]:
def _axis_angle_rotation(axis: str, angle: torch.Tensor) -> torch.Tensor:
    """
    Return the rotation matrices for one of the rotations about an axis
    of which Euler angles describe, for each value of the angle given.

    Args:
        axis: Axis label "X" or "Y or "Z".
        angle: any shape tensor of Euler angles in radians

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """

    cos = torch.cos(angle)
    sin = torch.sin(angle)
    one = torch.ones_like(angle)
    zero = torch.zeros_like(angle)

    if axis == "X":
        R_flat = (one, zero, zero, zero, cos, -sin, zero, sin, cos)
    elif axis == "Y":
        R_flat = (cos, zero, sin, zero, one, zero, -sin, zero, cos)
    elif axis == "Z":
        R_flat = (cos, -sin, zero, sin, cos, zero, zero, zero, one)
    else:
        raise ValueError("letter must be either X, Y or Z.")

    return torch.stack(R_flat, -1).reshape(angle.shape + (3, 3))

def euler_angles_to_matrix(euler_angles: torch.Tensor, convention: str) -> torch.Tensor:
    """
    Convert rotations given as Euler angles in radians to rotation matrices.

    Args:
        euler_angles: Euler angles in radians as tensor of shape (..., 3).
        convention: Convention string of three uppercase letters from
            {"X", "Y", and "Z"}.

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """
    if euler_angles.dim() == 0 or euler_angles.shape[-1] != 3:
        raise ValueError("Invalid input euler angles.")
    if len(convention) != 3:
        raise ValueError("Convention must have 3 letters.")
    if convention[1] in (convention[0], convention[2]):
        raise ValueError(f"Invalid convention {convention}.")
    for letter in convention:
        if letter not in ("X", "Y", "Z"):
            raise ValueError(f"Invalid letter {letter} in convention string.")
    matrices = [
        _axis_angle_rotation(c, e)
        for c, e in zip(convention, torch.unbind(euler_angles, -1))
    ]
    # return functools.reduce(torch.matmul, matrices)
    return torch.matmul(torch.matmul(matrices[0], matrices[1]), matrices[2])

In [ ]:
R.from_euler("xyz", (30,30,30), degrees=True).as_matrix()

In [ ]:
euler_angles_to_matrix(torch.tensor(np.radians(np.array((30,30,30)))), "ZYX")

In [ ]:
scalene main.py 
 22 files changed, 6285 insertions(+), 162 deletions(-)
 delete mode 100644 config/__init__.py